In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 22kB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 3.8MB 27.6MB/s 
     |████████████████████████████████| 512kB 51.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=d6ab2c754832ab3f5f0d485cc4eb0589fc3138d26ffdf750fc109a6b8fd0a756
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-use-rdr-4500-1-1-1.h5")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
import numpy as np
from numpy import argmax

In [ ]:
import re
def map_number_and_punct(word):
    # check hem va ngach
    num_of_seperate=0
    dem=0
    for char in word:
      if not char.isnumeric() and char!="/":
        dem+=1        
      if char=="/":
        num_of_seperate+=1
    if dem==0:
      if len(word)>1 and num_of_seperate==1:
        return u'<ngach>'
      if len(word)>1 and num_of_seperate>1:
        return u'<hem>'

    # if re.match(r"^[0-9]{2}0{3}$", word):
    #     return u'<postcode>'
    
    if word.isnumeric():
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'

    # word = word.replace("_"," ")
    
    return word

In [ ]:
import numpy as np
import codecs
def read_data(input_file):
    with codecs.open(input_file, 'r', 'utf-8') as f:
        word_list = [] 
        words = []
        for line in f:
            line = line.split()
            if len(line) > 0:
                words.append( map_number_and_punct(line[0].lower()[0:31]) )
            else:
                word_list.append(words)
                words = []
    return word_list

In [ ]:
test_word_sentences = read_data('/content/drive/MyDrive/fast_text/test/data_no_tag_pre.txt')
print(len(test_word_sentences))

900


In [ ]:
#get char embedding
def read_char_vocab(file_path):
  char_vocab = []
  for line in open(file_path, encoding='utf-8'):
    char_vocab.append(line.splitlines()[0])      
  return char_vocab

def char_to_int(char_vocab_data):
  dic = {}
  index = -1
  for char in char_vocab_data:
    try:
      dic[char]
    except:
      index = index + 1
      dic[char]=index
  return dic
 
def get_char_encode(sentence,max_length_of_a_sentence,max_length_of_a_word):
  char_vocab_data = read_char_vocab("/content/drive/MyDrive/fast_text/VISCII_short.txt")
  dic = char_to_int(char_vocab_data)
  # sentence  words is a sentence | ['i','am','an']
  sentence_encoded = np.zeros([max_length_of_a_sentence,max_length_of_a_word]) # 25*25
  for j in range(len(sentence)):  
    word = sentence[j].lower()
    # integer_encoded = [char_to_int[char] for char in word]
    word_encoded = np.zeros(max_length_of_a_word)
    for k in range(len(word)):
      char = word[k]
      try:
        word_encoded[k]= dic[char]
      except:
        # print("error : " + str(char)+" "+str(word)+" "+str(len(word)))
        word_encoded[k]= dic['[unk]']
    # sentence encoded
    sentence_encoded[j] = word_encoded
  return sentence_encoded

def get_charEmbedd_form_encode(charEnocde):
  LEN_OF_VOCAB = 137
  shape = charEnocde.shape
  char_embedd = np.zeros([shape[0],shape[1],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      char_int = charEnocde[i,j]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,:] = onehot
  return char_embedd

char_embedd = []
for sen_token in test_word_sentences:
  char_en = get_char_encode(sen_token,42,32)
  char_em = get_charEmbedd_form_encode(char_en)
  char_embedd.append(char_em)

char_embedd = np.array(char_embedd)
print(char_embedd.shape)

(900, 42, 32, 137)


In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/test/tag_embedd.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(900, 42, 13)


In [ ]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/test/word_embedd.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(900, 42, 300)


In [ ]:
y_pred = m.predict([char_embedd, word])
y_true = tag
print(y_pred.shape)
print(y_true.shape)

(900, 42, 13)
(900, 42, 13)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
print(y_pred.shape)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)
y_pred = y_pred.reshape(int(y_pred.shape[0]/42), 42)
print(y_pred.shape)

(37800, 13)
(37800,)
(900, 42)


In [ ]:
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
print(y_true.shape)
y_true = np.argmax(y_true,axis=1)
print(y_true.shape)
y_true = y_true.reshape(int(y_true.shape[0]/42), 42)
print(y_true.shape)

(37800, 13)
(37800,)
(900, 42)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def arr_tag_in_a_tag_sen(sen_pred, sen_true, tag):
  j=0
  bat_dau=-1
  ket_thuc=-1
  while j < len(sen_pred):
    if sen_true[j]==tag:
      bat_dau=j
      ket_thuc=j
      while sen_true[ket_thuc+1]==1:
        ket_thuc=ket_thuc+1
      ket_thuc=ket_thuc+1
      break
    else:
        j=j+1

  if bat_dau==-1 and ket_thuc==-1:
    return 0
  
  a = sen_pred[bat_dau:ket_thuc]
  b = sen_true[bat_dau:ket_thuc]
  a = np.array(a)
  b = np.array(b)
  if (a==b).all():
    return 1
  else:
    return 0

In [ ]:
from sklearn.metrics import accuracy_score
acc=0
false=0

homenumber_true = 0
street_true = 0 
ward_true = 0
district_true = 0
province_true = 0
country_true = 0
postcode_true = 0
ner_true = 0
obj_true = 0
obj_feature_true = 0
pre_true = 0

homenumber_false = 0
street_false = 0
ward_false = 0
district_false = 0
province_false = 0
country_false = 0
postcode_false = 0
ner_false = 0
obj_false = 0
obj_feature_false = 0
pre_false = 0

In [ ]:
for i in range(len(y_true)):
  sen_pred = y_pred[i]
  sen_true = y_true[i]
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 2)==1:
    street_true+=1
  else:
    street_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 3)==1:
    ward_true+=1
  else:
    ward_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 4)==1:
    district_true+=1
  else:
    district_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 5)==1:
    province_true+=1
  else:
    province_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 6)==1:
    country_true+=1
  else:
    country_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 7)==1:
    postcode_true+=1
  else:
    postcode_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 8)==1:
    ner_true+=1
  else:
    ner_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 9)==1:
    obj_true+=1
  else:
    obj_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 10)==1:
    obj_feature_true+=1
  else:
    obj_feature_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 11)==1:
    pre_true+=1
  else:
    pre_false+=1

In [ ]:
acc = homenumber_true+ street_true+ ward_true+ district_true+ province_true+ country_true+ postcode_true+ ner_true+ obj_true+ obj_feature_true+ pre_true
false = homenumber_false+ street_false+ ward_false+ district_false+ province_false+ country_false+ postcode_false+ ner_false+ obj_false+ obj_feature_false+ pre_false
print(acc)
print(false)
print('accuracy = {}'.format(acc/(acc+false)) )
ti_le_dung = format(acc/(acc+false)*100, '.2f')
print(ti_le_dung)

print("homenumber : {} true, {} false".format(homenumber_true, homenumber_false))
print("street : {} true, {} false".format(street_true, street_false))
print("ward : {} true, {} false".format(ward_true, ward_false))
print("district : {} true, {} false".format(district_true, district_false))
print("province : {} true, {} false".format(province_true, province_false))
print("country : {} true, {} false".format(country_true, country_false))
print("postcode : {} true, {} false".format(postcode_true, postcode_false))
print("ner : {} true, {} false".format(ner_true, ner_false))
print("obj : {} true, {} false".format(obj_true, obj_false))
print("obj_feature : {} true, {} false".format(obj_feature_true, obj_feature_false))
print("pre : {} true, {} false".format(pre_true, pre_false))

5710
5090
accuracy = 0.5287037037037037
52.87
homenumber : 984 true, 816 false
street : 587 true, 313 false
ward : 273 true, 627 false
district : 584 true, 316 false
province : 584 true, 316 false
country : 307 true, 593 false
postcode : 584 true, 316 false
ner : 293 true, 607 false
obj : 595 true, 305 false
obj_feature : 327 true, 573 false
pre : 592 true, 308 false


In [ ]:
dict = []
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1])
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1])
for element in y_true:
  if element not in dict:
    dict.append(element)
# print(dict)
dict.sort()
print(dict)

cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
for i in range(len(cnf_matrix)):
  for j in range(len(cnf_matrix)):
    cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
print(cnf_matrix)

arr_to_cal_predict = []
for i in range(len(cnf_matrix)):
  tong= 0
  for j in range(len(cnf_matrix)):
    tong+=cnf_matrix[j][i]
  arr_to_cal_predict.append(tong)
print(arr_to_cal_predict)
# lấy độ predict
predict = []
for i in range(len(cnf_matrix)):
  tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
  predict.append(tong)

print("Độ đo P của từng tag: ")
print(predict)
print(len(predict))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[[27027     0     0     0     0     0     0     0     8     0     0     0
     33]
 [    0   638    18     0     0     0     0     0     0     0     0    11
      0]
 [    0     4  2178    20    19     0     0     0     0     0     0     0
      0]
 [    0     0     0   273     4     0     0     0     0     0     0     0
      0]
 [    0     0     0     0   584    16     0     0     0     0     0     0
      0]
 [    0     0     0     0     0   584     6    10     0     0     0     0
      0]
 [    0     0     0     0     0     0   307     6     0     0     0     0
      0]
 [   15     0     0     0     0     0     0   584     0     0     0     0
      1]
 [    6     0     0     0     1     0     0     4  1001     4     0     5
      3]
 [    0     0     1     0     0     0     0     0     3  1128    25     8
      5]
 [    0     4     4     0     0     0     0     2    11    20   663    10
     13]
 [    0     1     3     0     0     0     0 

In [ ]:
normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
for i in range(len(normalized_confusion_matrix)):
  for j in range(len(normalized_confusion_matrix)):
    normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
print(normalized_confusion_matrix)

# lấy độ recall
recall = []
for i in range(len(normalized_confusion_matrix)):
  recall.append(normalized_confusion_matrix[i][i])

print("Độ đo R của từng tag: ")
print(recall)
print(len(recall))

[[0.998 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.001]
 [0.    0.957 0.027 0.    0.    0.    0.    0.    0.    0.    0.    0.016
  0.   ]
 [0.    0.002 0.981 0.009 0.009 0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.986 0.014 0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.973 0.027 0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.973 0.01  0.017 0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.981 0.019 0.    0.    0.    0.
  0.   ]
 [0.025 0.    0.    0.    0.    0.    0.    0.973 0.    0.    0.    0.
  0.002]
 [0.006 0.    0.    0.    0.001 0.    0.    0.004 0.978 0.004 0.    0.005
  0.003]
 [0.    0.    0.001 0.    0.    0.    0.    0.    0.003 0.964 0.021 0.007
  0.004]
 [0.    0.006 0.006 0.    0.    0.    0.    0.003 0.015 0.028 0.912 0.014
  0.018]
 [0.    0.001 0.003 0.    0.    0.    0.    0.    0.001 0.    0.    0.994
  0.001]
 [0.001 0.    0.    0.   

In [ ]:
# lấy độ F1
f1 = []
for i in range(len(recall)):
  p = predict[i]
  r = recall[i]
  f1.append(2*p*r/(p+r))

print("Độ đo F1 của từng tag: ")
print(f1)
print(len(f1))

Độ đo F1 của từng tag: 
[0.9985929780850736, 0.9713270743466124, 0.9845884587629293, 0.9581027161513299, 0.9667229223943162, 0.9731666381229662, 0.980915328155713, 0.9683258381204426, 0.9763389558024658, 0.9706881099204805, 0.9371199018021537, 0.9796589384624248, 0.9804668457867113]
13
